In [3]:
 # --- 配置路径 (请根据你的实际文件名修改) ---
BAM_FILE = "/data/work/human_brain/anno.bam"  # 50_2
GTF_FILE = "/data/work/human_brain/genes.gtf"
FASTA_FILE = "/data/work/human_brain/genome.fasta"
DB_FILE = "/data/work/hg38_annotations.db" # 运行后生成的数据库缓存

In [6]:
# 查看数据库中所有已定义的特征类型
print(list(db.featuretypes()))

['CDS', 'Selenocysteine', 'UTR', 'exon', 'gene', 'start_codon', 'stop_codon', 'transcript']


In [7]:
import pandas as pd

def verify_gtf_hierarchy(db, num_to_verify=3):
    """
    GTF文件中标注UTR，检验UTR坐标是否包含在exon坐标中
    3端5端UTR，第一个和最后一个exon根据gene方向链确定
    """
    print(f"--- 启动 GTF 坐标包含关系验证 (抽取 {num_to_verify} 个样本) ---")
    verified_count = 0
    
    # 遍历转录本，寻找含有 UTR 标签的记录
    for tx in db.features_of_type('transcript'):
        utrs = list(db.children(tx, featuretype='UTR'))
        exons = list(db.children(tx, featuretype='exon'))
        
        if not utrs or not exons:
            continue
            
        verified_count += 1
        strand = tx.strand
        
        # 1. 根据链方向找到物理位置上的 3' UTR
        # 正链：坐标最大的 UTR；负链：坐标最小的 UTR
        if strand == '+':
            three_p_utr = max(utrs, key=lambda x: x.end)
        else:
            three_p_utr = min(utrs, key=lambda x: x.start)
            
        # 2. 找到包含这个 3' UTR 的外显子
        # 理论上 3' UTR 必须落在某个 exon 范围内
        parent_exon = None
        for e in exons:
            if e.start <= three_p_utr.start and e.end >= three_p_utr.end:
                parent_exon = e
                break
        
        # 3. 验证转录本是否包含 UTR
        tx_contains_utr = (tx.start <= three_p_utr.start and tx.end >= three_p_utr.end)
        
        # 打印报告
        print(f"\n【样本 {verified_count}】 转录本 ID: {tx.id} (Strand: {strand})")
        print(f"  - 3' UTR 坐标: [{three_p_utr.start}, {three_p_utr.end}]")
        
        if parent_exon:
            print(f"  - 对应外显子 坐标: [{parent_exon.start}, {parent_exon.end}]")
            print(f"  - [验证 A] 外显子是否包含 UTR: ✅ 是")
        else:
            print(f"  - [验证 A] 外显子是否包含 UTR: ❌ 否 (未找到匹配外显子)")
            
        if tx_contains_utr:
            print(f"  - [验证 B] 转录本是否包含 UTR: ✅ 是")
        else:
            print(f"  - [验证 B] 转录本是否包含 UTR: ❌ 否")

        if verified_count >= num_to_verify:
            break

    if verified_count < num_to_verify:
        print(f"\n警告：仅在数据库中找到 {verified_count} 个符合条件的转录本。")

# 执行验证
verify_gtf_hierarchy(db)

--- 启动 GTF 坐标包含关系验证 (抽取 3 个样本) ---

【样本 1】 转录本 ID: ENST00000641515 (Strand: +)
  - 3' UTR 坐标: [70006, 71585]
  - 对应外显子 坐标: [69037, 71585]
  - [验证 A] 外显子是否包含 UTR: ✅ 是
  - [验证 B] 转录本是否包含 UTR: ✅ 是

【样本 2】 转录本 ID: ENST00000335137 (Strand: +)
  - 3' UTR 坐标: [70006, 70108]
  - 对应外显子 坐标: [69055, 70108]
  - [验证 A] 外显子是否包含 UTR: ✅ 是
  - [验证 B] 转录本是否包含 UTR: ✅ 是

【样本 3】 转录本 ID: ENST00000426406 (Strand: -)
  - 3' UTR 坐标: [450703, 450742]
  - 对应外显子 坐标: [450703, 451697]
  - [验证 A] 外显子是否包含 UTR: ✅ 是
  - [验证 B] 转录本是否包含 UTR: ✅ 是
